## THIS CELL HAS SOME JUNK WHICH I"LL DELETE LATER
Does the project describe what is the problem you are addressing in your project, what is the
relevant data, and how you can use this data to answer the question at hand?<br>
•
Does the project use techniques presented in the course (or clearly related to topics covered
in the course) to understand and analyze the data for this problem.<br>
•
Does the report situate the work clearly within the related work in this subject area.<br>
•
Does the report clearly demonstrate the work that was done, and provide directions for further
investigation.<br>

The final report will count for the majority of the total grade on the assignment (70%, based
only upon instructor feedback and not other student feedback), with the proposal adding an addi-
tional 5% (mainly just checking to ensure that groups made a reasonable effort to complete these
milestones), 20% for the video, and 5% for your comments on other students’ videos (again, just a
check that you made a reasonable effort).

# Predicting Building Energy Consumption Based on Building Characteristics

For our project we want to analyze energy consumption in commercial buildings.  We identified a dataset on the US Energy Information and Administration website called the CBECS Survey Data which contains information about energy consumption data for the year 2012.  It is a huge data set with 1121 features and 6721 building examples.

## __Step 1__.  Loading and Cleaning the Data.  
We are loading two data files.  The first file contains data of each building's energy use and characteristics.  The list of features given in the data set is so extensive, that there is a second file, which gives an explanation of each characteristics.  So this second file provides the order in which each buildng feature occurs in the data set, the abbreviation used, whether the feature is quantitative, or categorical and the unit of measurement.

In [1]:
import pandas as pd
import numpy as np

In [367]:
filepath = 'N:\\15-688\\HWFinalProject\\'
dataFile = '2012_public_use_data_aug2016.csv'
codeBk = '2012microdata_codebook.csv'

codeBook = pd.read_csv(filepath + codeBk, usecols=[0,1,5,6], index_col='Variable\nname')
codeDict = zip(codeBook.index, codeBook['Label'])
codeDict = {key:value for key,value in codeDict }  #creates a dictionary of column name and description

df = pd.read_csv(filepath + dataFile)
s = [a for a in df.dtypes if a.name !='int64' and a.name != 'float64'] #Check that all columns are of type ints or floats
assert s == []

df.dropna(subset=['ELCNS','ELHTBTU','ELCLBTU','ELVNBTU'], axis=0, inplace=True) #remove all rows where predicted values are NAN
df.index =  pd.RangeIndex(len(df))
ytotal = df['ELCNS']  # Annual electricity consumption (kWh)
yheat = df['ELHTBTU']
ycool = df['ELCLBTU']
yvent = df['ELVNBTU']
colsToDrop = list(range(850,1118))   #remove all useless columns, or ones with target values
colsToDrop.remove(1048)
colsToDrop.remove(1049)
df = df.drop(df.columns[colsToDrop],axis = 1)
print('size of data before cleaning:',df.shape)
print('\nnumber of null values before cleaning\n',df.isnull().sum().sort_values(ascending=False).head()) 
df = df.dropna(thresh = len(df)*11//12 , axis=1)    #remove columns where 1/12th or more of the data is missing

print('\nnumber of null values after cleaning\n',df.isnull().sum().sort_values(ascending=False).head())
print('Size of data after cleaing:' , df.shape,'Size of codeBook:', codeBook.shape)
print('RFCOOL =' ,codeDict['RFCOOL'])

size of data before cleaning: (6554, 853)

number of null values before cleaning
 COCOOK    6542
COMANU    6542
COWATR    6542
COOTH     6542
COGENR    6542
dtype: int64

number of null values after cleaning
 HWTRM     547
WTHTEQ    547
HTPMPC    518
CHWT      518
MAINCL    518
dtype: int64
Size of data after cleaing: (6554, 553) Size of codeBook: (1120, 3)
RFCOOL = Cool roof materials


In [368]:
codeBook[:5]

,File order,Label,Values/Format codes
Variable name,,,
PUBID,1.0,Building identifier,00001 - 06720
REGION,2.0,Census region,'1' = 'Northeast'\n'2' = 'Midwest'\n'3' = 'Sou...
CENDIV,3.0,Census division,'1' = 'New England'\n'2' = 'Middle Atlantic'\n...
PBA,4.0,Principal building activity,'01' = 'Vacant'\n'02' = 'Office'\n'04' = 'Labo...
FREESTN,5.0,Freestanding building,1' = 'Yes'\nMissing='No'


## Step 2.  Impute missing values

In [330]:
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values='NaN', strategy='median', axis=0)
imp.fit(df)
df = pd.DataFrame(data=imp.transform(df) , columns=df.columns)
df.isnull().sum().sort_values(ascending=False).head()

PUBCLIM    0
ZDRYCL     0
ZEDSEAT    0
ZPBSEAT    0
ZRWSEAT    0
dtype: int64

## Step 3.  Divide the Data for Validation

In [332]:
#reserving 1/4th data for validation as the test set
np.random.seed(12)
P = np.arange(df.shape[0])
np.random.shuffle(P)
size = len(df) *3//4
trainX = df.iloc[P[:size] ]
trainYtotal = ytotal[P[:size] ]
testX = df.iloc[P[size:]]
testYtotal = ytotal[P[size:]]

trainX.shape, trainY.shape, testX.shape, testYtotal.shape, trainYtotal.isnull().sum()

((4915, 553), (4915,), (1639, 553), (1639,), 0)

## Step 4.  Find Best Performing Features

In [347]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

sel = SelectKBest(f_regression, k=10)
sel.fit(trainX, trainYtotal)
x = sel.transform(trainX)


C:\Users\Noreen\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:298: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
C:\Users\Noreen\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:303: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\Noreen\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Noreen\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Noreen\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [372]:
s = zip(sel.scores_, df.columns)
d = {key: codeDict[value] for key, value in s  }
viewScores = pd.DataFrame(index= trainX.columns, data=sel.scores_, columns=['scores'])
viewScores['explanation'] = codeBook['Label']
viewScores.sort_values(by='scores', ascending=False)[:70]

,scores,explanation
SQFT,4202.733358,Square footage
NWKER,2686.371517,Number of employees
PCTERMN,2313.009694,Number of computers
NWKERC,1390.313259,Number of employees category
NOCC,1327.862970,Number of businesses
SQFTC,1280.106916,Square footage category
SERVERC,1279.860964,Number of servers category
PCTRMC,1212.746457,Number of computers category
ZHCBED,1074.179689,Imputed licensed bed capacity
LAPTPC,1023.378322,Number of laptops category


Questions
1.  How to fill in missing values
2.  Feature extraction
3.  

In [258]:
np.nan

nan